In [4]:
from Data_processing import import_images as i
from Classification import classification_class as classification
from Features_extraction import interest_point_detection as interest_point_detection
import pandas as pd

In [5]:
metrics_results_best_methods = {
        "accuracy" : {},
        "f1-score" : {},
        "recall" : {},
        "precision" : {},
        "roc_auc" : {}
        }

name_best_models = {}

# I. Point of interest detection

In [6]:
df_features = pd.DataFrame(interest_point_detection.method_SIFT(i.images))
df_Y = pd.DataFrame(i.Y)

data_SIFT = classification.DataProcessing(df_features, df_Y, stratified=False)
env_SIFT = classification.BinaryClassification(data_SIFT, average="macro")
metrics_results = env_SIFT.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_SIFT.createMeansDataframe(metrics_results, labels)
results_train_KFold.style.highlight_max(axis=0)

best_method_name = env_SIFT.get_best_method(results_test_KFold, "F1-score", ens="Test")
print(f"Best method name: {best_method_name}")
name_best_models["SIFT + BoW"] = best_method_name
metrics_results, predictions, models = env_SIFT.TrainTest()
env_SIFT.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_SIFT.get_metrics(models[best_method_name], "SIFT + BoW", metrics_results_best_methods)

=========1. Extract SIFT features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=========3. Cluster descriptors using KMeans (BoVW approach) =========
========= 4. Create feature histograms for each image =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.81      0.78        86
           1       0.62      0.52      0.57        50

    accuracy                           0.71       136
   macro avg       0.68      0.67      0.67       136
weighted avg       0.70      0.71      0.70       136



In [7]:
df_features = pd.DataFrame(interest_point_detection.method_ORB(i.images))
df_Y = pd.DataFrame(i.Y)

data_ORB = classification.DataProcessing(df_features, df_Y, stratified=False)
env_ORB = classification.BinaryClassification(data_ORB, average="macro")
metrics_results = env_ORB.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_ORB.createMeansDataframe(metrics_results, labels)
results_train_KFold.style.highlight_max(axis=0)

best_method_name = env_ORB.get_best_method(results_test_KFold, "F1-score", ens="Test")
print(f"Best method name: {best_method_name}")
name_best_models["ORB + BoW"] = best_method_name
metrics_results, predictions, models = env_ORB.TrainTest()
env_ORB.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_ORB.get_metrics(models[best_method_name], "ORB + BoW", metrics_results_best_methods)

=========1. Extract ORB features from all images=========
=========2. Stack all descriptors for clustering (BoVW) =========
=========3. Cluster descriptors using KMeans (BoVW approach) =========
========= 4. Create feature histograms for each image =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.48      0.55        86
           1       0.38      0.56      0.46        50

    accuracy                           0.51       136
   macro avg       0.52      0.52      0.50       136
weighted avg       0.55      0.51      0.52       136



# II. Edge detection

In [8]:
from Features_extraction import edge_detection as edges

df_features_edge = edges.features_df

data_edge = classification.DataProcessing(df_features_edge, df_Y, stratified=False)
env_edge = classification.BinaryClassification(data_edge, average="macro")
metrics_results = env_edge.CrossValidationKFold()
labels = list(metrics_results['f1-score'].keys())
results_train_KFold, results_test_KFold = env_edge.createMeansDataframe(metrics_results, labels)
results_train_KFold.style.highlight_max(axis=0)

best_method_name = env_edge.get_best_method(results_test_KFold, "F1-score", ens="Test")
print(f"Best method name: {best_method_name}")
name_best_models["EDGE"] = best_method_name
metrics_results, predictions, models = env_edge.TrainTest()
env_edge.evaluate_model(models[best_method_name])
metrics_results_best_methods = env_edge.get_metrics(models[best_method_name], "EDGE", metrics_results_best_methods)

metrics_results_best_methods

========= Extracting Edge Detection Features from Original Images =========
Optimization Logistic Regression
Optimization Classification Decision Tree
Optimization Random Forest
Best method name: LogReg
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.83      0.86        86
           1       0.74      0.84      0.79        50

    accuracy                           0.83       136
   macro avg       0.82      0.83      0.82       136
weighted avg       0.84      0.83      0.83       136



{'accuracy': {'SIFT + BoW': 0.7058823529411765,
  'SURF + BoW': 0.5073529411764706,
  'EDGE': 0.8308823529411765},
 'f1-score': {'SIFT + BoW': 0.6714975845410628,
  'SURF + BoW': 0.5028100616576635,
  'EDGE': 0.8228263947890115},
 'recall': {'SIFT + BoW': 0.6669767441860466,
  'SURF + BoW': 0.5183720930232558,
  'EDGE': 0.8327906976744186},
 'precision': {'SIFT + BoW': 0.6818642350557245,
  'SURF + BoW': 0.5171776473146337,
  'EDGE': 0.8177881412391739},
 'roc_auc': {'SIFT + BoW': 0.6669767441860466,
  'SURF + BoW': 0.5183720930232558,
  'EDGE': 0.8327906976744185}}

# III. Overall results

In [9]:
df_best_models_all_datasets_macro_f1 = pd.DataFrame(metrics_results_best_methods)
df_best_models_all_datasets_macro_f1.insert(0, 'Best models', name_best_models)
df_best_models_all_datasets_macro_f1.style.highlight_max(axis=0, subset=df_best_models_all_datasets_macro_f1.columns[1:])

,Best models,accuracy,f1-score,recall,precision,roc_auc
SIFT + BoW,LogReg,0.705882,0.671498,0.666977,0.681864,0.666977
SURF + BoW,LogReg,0.507353,0.502810,0.518372,0.517178,0.518372
EDGE,LogReg,0.830882,0.822826,0.832791,0.817788,0.832791
